<a href="https://colab.research.google.com/github/hamagami/preprocess/blob/main/04_08_PCA%E3%81%AB%E3%82%88%E3%82%8B%E6%AC%A1%E5%85%83%E5%89%8A%E6%B8%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCAによる次元削減

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.decomposition import PCA #主成分分析器
%matplotlib inline

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/maskot1977/ipython_notebook/master/toydata/wine.txt", sep="\t", index_col=0)

In [ ]:
df.head()

In [ ]:
df["class"].value_counts() # 1,2,3 という３つのclassがある

class 以外の変数で散布図行列をつくる。いくつかの２変数でclassの分布はわかれてみえるところもあるが、きれいに分けられるものはない

In [ ]:
from pandas import plotting 
plotting.scatter_matrix(df.iloc[:, 1:], figsize=(8, 8), c=list(df.iloc[:, 0]), alpha=0.5)
plt.show()

## オリジナルのデータと標準化をしたデータをつくる


In [ ]:
df_org = df.iloc[:,1:] #class を除いた説明変数だけのデータにする
df_org.head()

標準化を行っていないデータのままPCAをとり，その第一，第二主成分とclassの分布の関係をみてみる

In [ ]:
pca_org=PCA()
pca_org.fit(dforg)
feature_org = pca_org.transform(df_org)
plt.figure(figsize=(6, 6))
plt.scatter(feature_org[:, 0], feature_org[:, 1], alpha=0.8, c=list(df.iloc[:, 0]))
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

PCAの結果で散布図行列

In [ ]:
from pandas import plotting 
plotting.scatter_matrix(pd.DataFrame(feature_org, 
                        columns=["PC{}".format(x + 1) for x in range(len(df_org.columns))]), 
                        figsize=(8, 8), c=list(df.iloc[:, 0]), alpha=0.5) 
plt.show()

PCAの結果，主成分の情報をみてみる

In [ ]:
pca_org.explained_variance_ratio_
pd.DataFrame(feature_org, columns=["PC{}".format(x + 1) for x in range(len(df_org.columns))]).head()

累積寄与度　絶対値の大きい少数の変数の寄与が多いので必要な次数は少なくみえる

In [ ]:
import matplotlib.ticker as ticker
plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
plt.plot([0] + list( np.cumsum(pca_org.explained_variance_ratio_)), "-o")
plt.xlabel("Number of principal components")
plt.ylabel("Cumulative contribution rate")
plt.grid()
plt.show()

In [ ]:
# 第一主成分と第二主成分における観測変数の寄与度をプロットする
# Magnesium と　proline だけが大きく寄与しているようにみえる
plt.figure(figsize=(6, 6))
for x, y, name in zip(pca_org.components_[0], pca_org.components_[1], df.columns[1:]):
    plt.text(x, y, name)
plt.scatter(pca_org.components_[0], pca_org.components_[1], alpha=0.8)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# 標準化をした場合のPCA

In [ ]:
df_gen = df.iloc[:, 1:].apply(lambda x: (x-x.mean())/x.std(), axis=0)
df_gen.head()

In [ ]:
#主成分分析の実行
pca_gen = PCA()
pca_gen.fit(df_gen)
feature_gen = pca_gen.transform(df_gen)


In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(feature_gen[:, 0], feature_gen[:, 1], alpha=0.8, c=list(df.iloc[:, 0]))
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
from pandas import plotting 
plotting.scatter_matrix(pd.DataFrame(feature_gen, 
                        columns=["PC{}".format(x + 1) for x in range(len(df_gen.columns))]), 
                        figsize=(8, 8), c=list(df.iloc[:, 0]), alpha=0.5) 
plt.show()

In [ ]:
# 累積寄与率を図示する
import matplotlib.ticker as ticker
plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
plt.plot([0] + list( np.cumsum(pca_gen.explained_variance_ratio_)), "-o")
plt.xlabel("Number of principal components")
plt.ylabel("Cumulative contribution rate")
plt.grid()
plt.show()

In [ ]:
# 第一主成分と第二主成分における観測変数の寄与度をプロットする
plt.figure(figsize=(6, 6))
for x, y, name in zip(pca_gen.components_[0], pca_gen.components_[1], df.columns[1:]):
    plt.text(x, y, name)
plt.scatter(pca_gen.components_[0], pca_gen.components_[1], alpha=0.8)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
df.loc[:,['class','Magnesium','Proline']]